In [44]:
import pandas as pd 
import os
import random

In [45]:
repeat0 = 10
repeat1 = 1000

In [46]:
code = rf"""

; ------------------------------------
; Author: Hosein Yavarzadeh 
; Email: hyavarzadeh@ucsd.edu
; Supervisor: Prof. Dean Tullsen
; ------------------------------------

%define repeat0  {repeat0}             ; Number of repetitions of whole test. Default = 8
%define repeat1  {repeat1}           ; Repeat count for loop around testcode. Default = no loop
%define repeat2  1              ; Repeat count for repeat macro around testcode. Default = 100

%macro testinitc 0              ; Macro to call in each test before reading counters.
    mov dword[PrintCustomPMC], 1
    mov ebx, 8000             
    lea rdi, [UserData]
    rand_loop:
    rdrand eax
    and eax, 1
    mov [rdi], al
    inc rdi
    dec ebx
    jnz rand_loop

%endmacro

%macro testinit2 0              ; Macro with initializations before each test. Default = nothing
    lea rdi, [UserData]
%endmacro

%macro testcode 0               ; A multi-line macro executing any piece of test code.

    inc rdi

    cmp byte[rdi],0
    
    ; dummy_branch 300, 0

    ; jmp rand_branch
    mov rax, 0x100000000
    mov rbx, rand_branch_back
    mov rdx, 200
    mov rcx, 0
    cmp rcx, 0
    jmp rax
    rand_branch_back:

    dummy_branch var, 5

    ; test branch
    READ_PMC_START
    cmp byte[rdi],0
    je target2
    target2:
    READ_PMC_END

%endmacro

%macro dummy_branch 2
      mov rax, 0
      %assign i (100)*(%2)
      %assign j (100)*(%2)
      %rep %1
            %assign j i+1
            label%+ i:
            cmp rax, 0
            je label%+ j 
            %assign i j  
      %endrep
    ;   align (1 << my_align)
      label%+ j:
%endmacro

"""

In [47]:
ld_script = r"""
/* Script for -z combreloc -z separate-code */
/* Copyright (C) 2014-2020 Free Software Foundation, Inc.
   Copying and distribution of this script, with or without modification,
   are permitted in any medium without royalty provided the copyright
   notice and this notice are preserved.  */
OUTPUT_FORMAT("elf64-x86-64", "elf64-x86-64",
	      "elf64-x86-64")
OUTPUT_ARCH(i386:x86-64)
ENTRY(_start)
SEARCH_DIR("=/usr/local/lib/x86_64-linux-gnu"); SEARCH_DIR("=/lib/x86_64-linux-gnu"); SEARCH_DIR("=/usr/lib/x86_64-linux-gnu"); SEARCH_DIR("=/usr/lib/x86_64-linux-gnu64"); SEARCH_DIR("=/usr/local/lib64"); SEARCH_DIR("=/lib64"); SEARCH_DIR("=/usr/lib64"); SEARCH_DIR("=/usr/local/lib"); SEARCH_DIR("=/lib"); SEARCH_DIR("=/usr/lib"); SEARCH_DIR("=/usr/x86_64-linux-gnu/lib64"); SEARCH_DIR("=/usr/x86_64-linux-gnu/lib");
SECTIONS
{
  PROVIDE (__executable_start = SEGMENT_START("text-segment", 0x400000)); . = SEGMENT_START("text-segment", 0x400000) + SIZEOF_HEADERS;
  .interp         : { *(.interp) }
  .note.gnu.build-id  : { *(.note.gnu.build-id) }
  .hash           : { *(.hash) }
  .gnu.hash       : { *(.gnu.hash) }
  .dynsym         : { *(.dynsym) }
  .dynstr         : { *(.dynstr) }
  .gnu.version    : { *(.gnu.version) }
  .gnu.version_d  : { *(.gnu.version_d) }
  .gnu.version_r  : { *(.gnu.version_r) }
  .rela.dyn       :
    {
      *(.rela.init)
      *(.rela.text .rela.text.* .rela.gnu.linkonce.t.*)
      *(.rela.fini)
      *(.rela.rodata .rela.rodata.* .rela.gnu.linkonce.r.*)
      *(.rela.data .rela.data.* .rela.gnu.linkonce.d.*)
      *(.rela.tdata .rela.tdata.* .rela.gnu.linkonce.td.*)
      *(.rela.tbss .rela.tbss.* .rela.gnu.linkonce.tb.*)
      *(.rela.ctors)
      *(.rela.dtors)
      *(.rela.got)
      *(.rela.bss .rela.bss.* .rela.gnu.linkonce.b.*)
      *(.rela.ldata .rela.ldata.* .rela.gnu.linkonce.l.*)
      *(.rela.lbss .rela.lbss.* .rela.gnu.linkonce.lb.*)
      *(.rela.lrodata .rela.lrodata.* .rela.gnu.linkonce.lr.*)
      *(.rela.ifunc)
    }
  .rela.plt       :
    {
      *(.rela.plt)
      PROVIDE_HIDDEN (__rela_iplt_start = .);
      *(.rela.iplt)
      PROVIDE_HIDDEN (__rela_iplt_end = .);
    }
  . = ALIGN(CONSTANT (MAXPAGESIZE));
  .init           :
  {
    KEEP (*(SORT_NONE(.init)))
  }
  .plt            : { *(.plt) *(.iplt) }
.plt.got        : { *(.plt.got) }
.plt.sec        : { *(.plt.sec) }
  . = 0x40000000;
  .text           :
  {
    b64.o (.text .stub .text.* .gnu.linkonce.t.*)
    *(.text.unlikely .text.*_unlikely .text.unlikely.*)
    *(.text.exit .text.exit.*)
    *(.text.startup .text.startup.*)
    *(.text.hot .text.hot.*)
    *(SORT(.text.sorted.*))
    a64.o (.text .stub .text.* .gnu.linkonce.t.*)
    /* .gnu.warning sections are handled specially by elf.em.  */
    *(.gnu.warning)
  }
  .fini           :
  {
    KEEP (*(SORT_NONE(.fini)))
  }
  PROVIDE (__etext = .);
  PROVIDE (_etext = .);
  PROVIDE (etext = .);
  . = ALIGN(CONSTANT (MAXPAGESIZE));
  /* Adjust the address for the rodata segment.  We want to adjust up to
     the same address within the page on the next page up.  */
  . = SEGMENT_START("rodata-segment", ALIGN(CONSTANT (MAXPAGESIZE)) + (. & (CONSTANT (MAXPAGESIZE) - 1)));
  .rodata         : { *(.rodata .rodata.* .gnu.linkonce.r.*) }
  .rodata1        : { *(.rodata1) }
  .eh_frame_hdr   : { *(.eh_frame_hdr) *(.eh_frame_entry .eh_frame_entry.*) }
  .eh_frame       : ONLY_IF_RO { KEEP (*(.eh_frame)) *(.eh_frame.*) }
  .gcc_except_table   : ONLY_IF_RO { *(.gcc_except_table .gcc_except_table.*) }
  .gnu_extab   : ONLY_IF_RO { *(.gnu_extab*) }
  /* These sections are generated by the Sun/Oracle C++ compiler.  */
  .exception_ranges   : ONLY_IF_RO { *(.exception_ranges*) }
  /* Adjust the address for the data segment.  We want to adjust up to
     the same address within the page on the next page up.  */
  . = DATA_SEGMENT_ALIGN (CONSTANT (MAXPAGESIZE), CONSTANT (COMMONPAGESIZE));
  /* Exception handling  */
  .eh_frame       : ONLY_IF_RW { KEEP (*(.eh_frame)) *(.eh_frame.*) }
  .gnu_extab      : ONLY_IF_RW { *(.gnu_extab) }
  .gcc_except_table   : ONLY_IF_RW { *(.gcc_except_table .gcc_except_table.*) }
  .exception_ranges   : ONLY_IF_RW { *(.exception_ranges*) }
  /* Thread Local Storage sections  */
  .tdata	  :
   {
     PROVIDE_HIDDEN (__tdata_start = .);
     *(.tdata .tdata.* .gnu.linkonce.td.*)
   }
  .tbss		  : { *(.tbss .tbss.* .gnu.linkonce.tb.*) *(.tcommon) }
  .preinit_array    :
  {
    PROVIDE_HIDDEN (__preinit_array_start = .);
    KEEP (*(.preinit_array))
    PROVIDE_HIDDEN (__preinit_array_end = .);
  }
  .init_array    :
  {
    PROVIDE_HIDDEN (__init_array_start = .);
    KEEP (*(SORT_BY_INIT_PRIORITY(.init_array.*) SORT_BY_INIT_PRIORITY(.ctors.*)))
    KEEP (*(.init_array EXCLUDE_FILE (*crtbegin.o *crtbegin?.o *crtend.o *crtend?.o ) .ctors))
    PROVIDE_HIDDEN (__init_array_end = .);
  }
  .fini_array    :
  {
    PROVIDE_HIDDEN (__fini_array_start = .);
    KEEP (*(SORT_BY_INIT_PRIORITY(.fini_array.*) SORT_BY_INIT_PRIORITY(.dtors.*)))
    KEEP (*(.fini_array EXCLUDE_FILE (*crtbegin.o *crtbegin?.o *crtend.o *crtend?.o ) .dtors))
    PROVIDE_HIDDEN (__fini_array_end = .);
  }
  .ctors          :
  {
    /* gcc uses crtbegin.o to find the start of
       the constructors, so we make sure it is
       first.  Because this is a wildcard, it
       doesn't matter if the user does not
       actually link against crtbegin.o; the
       linker won't look for a file to match a
       wildcard.  The wildcard also means that it
       doesn't matter which directory crtbegin.o
       is in.  */
    KEEP (*crtbegin.o(.ctors))
    KEEP (*crtbegin?.o(.ctors))
    /* We don't want to include the .ctor section from
       the crtend.o file until after the sorted ctors.
       The .ctor section from the crtend file contains the
       end of ctors marker and it must be last */
    KEEP (*(EXCLUDE_FILE (*crtend.o *crtend?.o ) .ctors))
    KEEP (*(SORT(.ctors.*)))
    KEEP (*(.ctors))
  }
  .dtors          :
  {
    KEEP (*crtbegin.o(.dtors))
    KEEP (*crtbegin?.o(.dtors))
    KEEP (*(EXCLUDE_FILE (*crtend.o *crtend?.o ) .dtors))
    KEEP (*(SORT(.dtors.*)))
    KEEP (*(.dtors))
  }
  .jcr            : { KEEP (*(.jcr)) }
  .data.rel.ro : { *(.data.rel.ro.local* .gnu.linkonce.d.rel.ro.local.*) *(.data.rel.ro .data.rel.ro.* .gnu.linkonce.d.rel.ro.*) }
  .dynamic        : { *(.dynamic) }
  .got            : { *(.got) *(.igot) }
  . = DATA_SEGMENT_RELRO_END (SIZEOF (.got.plt) >= 24 ? 24 : 0, .);
  .got.plt        : { *(.got.plt) *(.igot.plt) }
  .data           :
  {
    *(.data .data.* .gnu.linkonce.d.*)
    SORT(CONSTRUCTORS)
  }
  .data1          : { *(.data1) }
  _edata = .; PROVIDE (edata = .);
  . = .;
  __bss_start = .;
  .bss            :
  {
   *(.dynbss)
   *(.bss .bss.* .gnu.linkonce.b.*)
   *(COMMON)
   /* Align here to ensure that the .bss section occupies space up to
      _end.  Align after .bss to ensure correct alignment even if the
      .bss section disappears because there are no input sections.
      FIXME: Why do we need it? When there is no .bss section, we do not
      pad the .data section.  */
   . = ALIGN(. != 0 ? 64 / 8 : 1);
  }
  .lbss   :
  {
    *(.dynlbss)
    *(.lbss .lbss.* .gnu.linkonce.lb.*)
    *(LARGE_COMMON)
  }
  . = ALIGN(64 / 8);
  . = SEGMENT_START("ldata-segment", .);
  .lrodata   ALIGN(CONSTANT (MAXPAGESIZE)) + (. & (CONSTANT (MAXPAGESIZE) - 1)) :
  {
    *(.lrodata .lrodata.* .gnu.linkonce.lr.*)
  }
  .ldata   ALIGN(CONSTANT (MAXPAGESIZE)) + (. & (CONSTANT (MAXPAGESIZE) - 1)) :
  {
    *(.ldata .ldata.* .gnu.linkonce.l.*)
    . = ALIGN(. != 0 ? 64 / 8 : 1);
  }
  . = ALIGN(64 / 8);
  _end = .; PROVIDE (end = .);
  . = DATA_SEGMENT_END (.);
  /* Stabs debugging sections.  */
  .stab          0 : { *(.stab) }
  .stabstr       0 : { *(.stabstr) }
  .stab.excl     0 : { *(.stab.excl) }
  .stab.exclstr  0 : { *(.stab.exclstr) }
  .stab.index    0 : { *(.stab.index) }
  .stab.indexstr 0 : { *(.stab.indexstr) }
  .comment       0 : { *(.comment) }
  .gnu.build.attributes : { *(.gnu.build.attributes .gnu.build.attributes.*) }
  /* DWARF debug sections.
     Symbols in the DWARF debugging sections are relative to the beginning
     of the section so we begin them at 0.  */
  /* DWARF 1 */
  .debug          0 : { *(.debug) }
  .line           0 : { *(.line) }
  /* GNU DWARF 1 extensions */
  .debug_srcinfo  0 : { *(.debug_srcinfo) }
  .debug_sfnames  0 : { *(.debug_sfnames) }
  /* DWARF 1.1 and DWARF 2 */
  .debug_aranges  0 : { *(.debug_aranges) }
  .debug_pubnames 0 : { *(.debug_pubnames) }
  /* DWARF 2 */
  .debug_info     0 : { *(.debug_info .gnu.linkonce.wi.*) }
  .debug_abbrev   0 : { *(.debug_abbrev) }
  .debug_line     0 : { *(.debug_line .debug_line.* .debug_line_end) }
  .debug_frame    0 : { *(.debug_frame) }
  .debug_str      0 : { *(.debug_str) }
  .debug_loc      0 : { *(.debug_loc) }
  .debug_macinfo  0 : { *(.debug_macinfo) }
  /* SGI/MIPS DWARF 2 extensions */
  .debug_weaknames 0 : { *(.debug_weaknames) }
  .debug_funcnames 0 : { *(.debug_funcnames) }
  .debug_typenames 0 : { *(.debug_typenames) }
  .debug_varnames  0 : { *(.debug_varnames) }
  /* DWARF 3 */
  .debug_pubtypes 0 : { *(.debug_pubtypes) }
  .debug_ranges   0 : { *(.debug_ranges) }
  /* DWARF Extension.  */
  .debug_macro    0 : { *(.debug_macro) }
  .debug_addr     0 : { *(.debug_addr) }
  .gnu.attributes 0 : { KEEP (*(.gnu.attributes)) }
  /DISCARD/ : { *(.note.GNU-stack) *(.gnu_debuglink) *(.gnu.lto_*) }

"""

In [48]:
def create_nasm(cnt):
    nasm_new_sections = ''
    for i in range(cnt):
        nasm_new_sections += f"""
SECTION .dummy_branch_{i} exec
    dummy_label_{i}:
    je dummy_label_{i+1}

"""
    nasm_new_sections += f"""
SECTION .dummy_branch_{cnt} exec
    dummy_label_{cnt}:
    dec rdx
    cmp rdx, 0
    jmp dummy_label_{cnt+1}

SECTION .dummy_branch_{cnt+1} exec
    dummy_label_{cnt+1}:
    je dummy_label_{cnt+2}
    mov rax, 0x100000000
    mov rcx, 0
    cmp rcx, 0
    jmp rax

SECTION .dummy_branch_{cnt+2} exec
    dummy_label_{cnt+2}:
    cmp byte[rdi],0
    jmp dummy_label_{cnt+3}

SECTION .dummy_branch_{cnt+3} exec
    dummy_label_{cnt+3}:
    je dummy_label_{cnt+4}
    jmp dummy_label_{cnt+4}

SECTION .dummy_branch_{cnt+4} exec
    dummy_label_{cnt+4}:
    jmp rbx
"""
    with open("bp.nasm", "w") as file:
      file.write(code + nasm_new_sections)

In [49]:
def create_ld(a, b, cnt, stride=0x10000000, offset=0x100000000, flip_mask=0x0, crit_bits=0x0):
  ld_new_sections = ''

  for i in range(cnt):
    ld_new_sections += f"""
    . = {hex(offset + i * stride)};
    .dummy_branch_{i} : {{ *(.dummy_branch_{i}) }}
  """

  # ld_new_sections += f"""
  #   . = {hex(0x100000000 + cnt * stride + ((1 << a) if a >= 0 else 0))};
  #   .dummy_branch_{cnt} : {{ *(.dummy_branch_{cnt}) }}
  # """

  # ld_new_sections += f"""
  #   . = {hex(0x100000000 + (cnt + 1) * stride + ((b << b) if a >= 0 else 0))};
  #   .dummy_branch_{cnt + 1} : {{ *(.dummy_branch_{cnt + 1}) }}
  # """
  
  # b_addr = 0x100000000 + cnt * stride + random.randint(0, ((1 << a) if a >= 0 else 0)) & 0xffffffffdf # 0x20
  # b_addr = 0x100000000 + cnt * stride + 0x315a2fb ^ flip_mask
  b_addr = 0x100000000 + cnt * stride
  b_addr = b_addr | crit_bits
  # b_addr = b_addr ^ flip_mask
  # b_addr = 0x44315a2fb
  # b_addr = 0x440000038
  
  
  
  ld_new_sections += f"""
    . = {hex(b_addr)};
    .dummy_branch_{cnt} : {{ *(.dummy_branch_{cnt}) }}
  """
  
  t_addr = 0x100000000 + (cnt + 1) * stride + random.randint(0, ((1 << b) if b >= 0 else 0)) # 0x20
  # t_addr = 0x482f2fbe4

  ld_new_sections += f"""
    . = {hex(t_addr)};
    .dummy_branch_{cnt + 1} : {{ *(.dummy_branch_{cnt + 1}) }}
  """

  ld_new_sections += '}'

  with open("script.ld", "w") as file:
      file.write(ld_script + ld_new_sections)
      
  return b_addr, t_addr

In [50]:
# rdx 5
cnt = 10
start = 10
end = 10

i = 26
j = 26

final_dfs = []
for l in range(8,31):
    final_df = []
    for k in range(64):
        create_nasm(cnt)
        b_addr, t_addr = create_ld(i, j, cnt + 3, 1 << l, 0x100000000, 0x0, k)
        os.system(f"./run.sh {start} {end}")
        # if sample_cnt % 50 == 0:
        #     print(sample_cnt)

        data = pd.read_csv("results.csv", comment='#', delim_whitespace=True)
        data["Clock"] = pd.to_numeric(data["Clock"],errors='coerce')
        data["BrRetired"] = pd.to_numeric(data["BrRetired"],errors='coerce')
        data["BrMispred"] = pd.to_numeric(data["BrMispred"],errors='coerce')
        data["Clock"] = (data["Clock"]) / repeat1
        data["BrRetired"] = data["BrRetired"] / repeat1
        data["BrMispred"] = data["BrMispred"] / repeat1
        final_df.append((data.groupby(data.index // repeat0).mean()["BrMispred"][0], b_addr, t_addr))
    final_dfs.append(final_df)

In [51]:
for ct, df in enumerate(final_dfs, 8):
    print(ct, len([(m, b, t) for (m, b, t) in df if m < 0.25]))

8 64
9 64
10 64
11 64
12 64
13 64
14 64
15 64
16 64
17 64
18 64
19 64
20 5
21 5
22 5
23 59
24 31
25 32
26 13
27 6
28 5
29 5
30 5


8 64
9 64
10 64
11 64
12 64
13 64
14 64
15 64
16 64
17 64
18 64
19 64
20 5
21 5
22 5
23 58
24 32
25 30
26 13
27 6
28 5
29 5

In [52]:
for m, b, t in final_dfs[1]:
    if m < 0.25:
        print((0, hex(b), hex(t)))
    # else:
    #     print((m, hex(b), hex(t)))

(0, '0x100001a00', '0x103203c0f')
(0, '0x100001a01', '0x1026bb16b')
(0, '0x100001a02', '0x10216dd10')
(0, '0x100001a03', '0x101ea1da3')
(0, '0x100001a04', '0x103e7754a')
(0, '0x100001a05', '0x102e8f8fc')
(0, '0x100001a06', '0x100399e3b')
(0, '0x100001a07', '0x10163ec64')
(0, '0x100001a08', '0x1014ad93e')
(0, '0x100001a09', '0x103277bae')
(0, '0x100001a0a', '0x102022f72')
(0, '0x100001a0b', '0x1037d66b6')
(0, '0x100001a0c', '0x10223da25')
(0, '0x100001a0d', '0x1014d92c5')
(0, '0x100001a0e', '0x102bce8c8')
(0, '0x100001a0f', '0x10017920a')
(0, '0x100001a10', '0x102e049f3')
(0, '0x100001a11', '0x10364db4e')
(0, '0x100001a12', '0x1006ca857')
(0, '0x100001a13', '0x1005318f3')
(0, '0x100001a14', '0x10003205b')
(0, '0x100001a15', '0x1006efc44')
(0, '0x100001a16', '0x1035be490')
(0, '0x100001a17', '0x1003a5891')
(0, '0x100001a18', '0x10094ff13')
(0, '0x100001a19', '0x103b225d9')
(0, '0x100001a1a', '0x100554d74')
(0, '0x100001a1b', '0x10070095c')
(0, '0x100001a1c', '0x1037c7aad')
(0, '0x100001a

In [57]:
my_df = pd.DataFrame(final_dfs)

In [58]:
my_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,"(0.0004, 4294970624, 4314680293)","(0.0007, 4294970625, 4326390057)","(0.00030000000000000003, 4294970626, 4316566155)","(0.0008, 4294970627, 4350101881)","(0.0005, 4294970628, 4316205782)","(0.00030000000000000003, 4294970629, 4321050631)","(0.0006000000000000001, 4294970630, 4322629595)","(0.0004, 4294970631, 4332896534)","(0.0005, 4294970632, 4315948025)","(0.0006000000000000001, 4294970633, 4356980129)",...,"(0.0004, 4294970678, 4356008102)","(0.00030000000000000003, 4294970679, 4323338409)","(0.00030000000000000003, 4294970680, 4323585038)","(0.0006000000000000001, 4294970681, 4333496308)","(0.0008, 4294970682, 4299344827)","(0.0005, 4294970683, 4327637443)","(0.0005, 4294970684, 4306993477)","(0.0006000000000000001, 4294970685, 4357721876)","(0.0005, 4294970686, 4356766550)","(0.0004, 4294970687, 4359924717)"
1,"(0.0004, 4294973952, 4347411471)","(0.00030000000000000003, 4294973953, 4335579499)","(0.0005, 4294973954, 4330020112)","(0.0005, 4294973955, 4327087523)","(0.00030000000000000003, 4294973956, 4360467786)","(0.0005, 4294973957, 4343789820)","(0.0005, 4294973958, 4298743355)","(0.0005, 4294973959, 4318293092)","(0.00030000000000000003, 4294973960, 4316649790)","(0.0008, 4294973961, 4347886510)",...,"(0.0005, 4294974006, 4361586633)","(0.00030000000000000003, 4294974007, 4358626737)","(0.00030000000000000003, 4294974008, 4318470489)","(0.0004, 4294974009, 4305459589)","(0.0005, 4294974010, 4296700010)","(0.00030000000000000003, 4294974011, 4310021076)","(0.0007, 4294974012, 4337934516)","(0.00030000000000000003, 4294974013, 4343888230)","(0.0005, 4294974014, 4307920743)","(0.0004, 4294974015, 4361634126)"
2,"(0.00030000000000000003, 4294980608, 4307039490)","(0.00030000000000000003, 4294980609, 4323136541)","(0.0006000000000000001, 4294980610, 4361632687)","(0.0005, 4294980611, 4353675733)","(0.0005, 4294980612, 4346991325)","(0.0004, 4294980613, 4360794294)","(0.00030000000000000003, 4294980614, 4334382169)","(0.0004, 4294980615, 4338039675)","(0.00030000000000000003, 4294980616, 4304594276)","(0.0004, 4294980617, 4306360285)",...,"(0.0004, 4294980662, 4302903361)","(0.0008, 4294980663, 4324766623)","(0.0005, 4294980664, 4332998050)","(0.00030000000000000003, 4294980665, 4324554720)","(0.00030000000000000003, 4294980666, 4335369816)","(0.0004, 4294980667, 4348689327)","(0.0005, 4294980668, 4315341087)","(0.0005, 4294980669, 4355832393)","(0.0005, 4294980670, 4318479994)","(0.0004, 4294980671, 4344309450)"
3,"(0.0004, 4294993920, 4317185990)","(0.00030000000000000003, 4294993921, 4309563081)","(0.00030000000000000003, 4294993922, 4335123011)","(0.00030000000000000003, 4294993923, 4296155041)","(0.00030000000000000003, 4294993924, 4311664871)","(0.00030000000000000003, 4294993925, 4342037470)","(0.0005, 4294993926, 4346601229)","(0.00030000000000000003, 4294993927, 4335052973)","(0.00030000000000000003, 4294993928, 4324196755)","(0.0006000000000000001, 4294993929, 4320730854)",...,"(0.00030000000000000003, 4294993974, 4344726365)","(0.0004, 4294993975, 4322376944)","(0.00030000000000000003, 4294993976, 4326393876)","(0.00030000000000000003, 4294993977, 4299225631)","(0.00030000000000000003, 4294993978, 4315359501)","(0.0005, 4294993979, 4333003853)","(0.00030000000000000003, 4294993980, 4307578628)","(0.0006000000000000001, 4294993981, 4358528157)","(0.00030000000000000003, 4294993982, 4325452670)","(0.00030000000000000003, 4294993983, 4316629668)"
4,"(0.0004, 4295020544, 4358357198)","(0.0006000000000000001, 4295020545, 4338592090)","(0.00030000000000000003, 4295020546, 4329751633)","(0.00030000000000000003, 4295020547, 4313841681)","(0.00030000000000000003, 4295020548, 4333686461)","(0.00030000000000000003, 4295020549, 4361452657)","(0.00030000000000000003, 4295020550, 4318803478)","(0.00030000000000000003, 4295020551, 4340537104)","(0.0004, 4295020552, 4349949123)","(0.0005, 4295020553, 4355894137)",...,"(0.0004, 4295020598, 4351916511)","(0.00030000000000

In [55]:
hex(18305360635)

'0x44315a2fb'

In [56]:
my_df.to_pickle('./script11/1.pkl')